# MA12002 Programming Coursework

* **Set:** Fri 24th Nov 2023
* **Due:** Fri 8th Dec 2023, 12 noon
* **Total marks:** 40

This coursework assignment is worth 20\% of the total mark of MA12002.

<hr style="height: 2px">

**You must not discuss your work for this coursework assignment with anyone else. The work which
you hand in must be your own. Under no circumstances should you exchange solutions with other students. After you hand your work in, you could be asked by an examiner to explain it verbally as part of the marking process. Any evidence of cheating will entail disciplinary procedures and the risk of losing all points on the assignment for all parties involved. See [the official guidance](http://www.bath.ac.uk/library/help/infoguides/plagiarism.html) and [the QA53 document](http://www.bath.ac.uk/quality/documents/QA53.pdf) for further details.**

**Regarding the use of AI tools this coursework assignment falls into category A: you are <u>not</u> allowed to use tools such as ChatGPT when solving this coursework assignment.**

<hr style="height: 2px">

*&#169; Eike Mueller, University of Bath 2019-2023. This coursework description is copyright of Eike Mueller, University of Bath. It is provided exclusively for educational purposes at the University and is to be downloaded or copied for your private study only. Further distribution, e.g. by upload to external repositories, is prohibited.* 



## Background
### A number game
$n$ positive integer numbers $a_0,a_1,a_2,\dots,a_{n-1}\in\mathbb{N}$ are arranged in a circle as shown in Figure 1 below. Each number lies in the range $2,\dots,m$ for some positive integer $m\in \mathbb{N}$: for all $j\in\{0,1,\dots,n-1\}$ we have that $2\le a_j\le m$.

Consider the following game: starting at some position $k_0\in\{0,1,\dots,n-1\}$ which we are free to choose, we can move around the circle in the clockwise direction. If at some point in this process we are at the number $a_k$, we can either take a step of size $1$ without receiving any reward, or take a step of size $a_k$ and receive a reward of $a_k^2$. The process is repeated until we return to the position $k_0$ while moving around the circle exactly once in $S$ steps (this is only possible if the sum of the stepsizes adds up to $n$). For each way of going around the circle in this way we are interested in the total reward $R$ which is defined as

$$
R = r_1+r_2+\dots+r_S = \sum_{\sigma=1}^{S} r_\sigma\qquad\text{where $r_\sigma\in \{0,2^2,3^2,\dots,m^2\}$ is the award received in step $\sigma$.}
$$

![Two paths circular](figures/paths_circular.svg)
*Figure 1.: Two ways of moving around a circle of $n=8$ numbers with $m=4$. The red solution shown on the right will result in the maximum possible reward.*

In this coursework assignment we will implement algorithms for computing the maximum possible reward $R_{\max}(a)$ that can be achieved by playing this game for a given list of numbers $a=[a_0,a_1,a_2,\dots,a_{n-1}]$. Note that the starting point is not fixed: we are interested in the maximum award over all possible choices of $k_0\in\{0,1,2,\dots,n-1\}$.

### Non-circular variant
We first consider a modified, non-circular version of the game: assuming that we have $n$ numbers $b_0,b_1,b_2,\dots,b_{n-1}$ with $b_j\in\{2,3,\dots,m\}$ for all $j=0,1,\dots,n-1$, we always start at position $k_0=0$. Assuming that we are at position $k$ at some point in the process, we can move forward either by taking a step of size $1$ for a reward of zero or we can take a step of size $b_k$ for a reward of size $b_k^2$. The game ends if we finish exactly one step after position $n-1$, as shown in Figure 2 below. As above, the total reward $\widehat{R}$ is the sum over the awards in all steps.

Again we are interested in the maximum reward $\widehat{R}_{\max}(b)$ that can be achieved by playing the game for a given list of numbers $b=[b_0,b_1,b_2,\dots,b_{n-1}]$.

![Two paths linear](figures/paths_linear.svg)
*Figure 2.: Two ways of traversing a line of $n=7$ numbers with $m=5$. The blue solution shown at the top will result in the maximum possible reward.*

### Brute force solution
We first consider the non-circular version of the game. In this case, it is possible to compute the maximum reward recursively by using the following observation: the maximum award $\widehat{R}_{\max}([b_0,b_1,\dots,b_{n-1}])$ for the list $[b_0,b_1,\dots,b_{n-1}]$ with $b_0=\nu\in\{2,3,\dots,m\}$ is the larger of the following two numbers:
$$
\begin{aligned}
r_{1} &= \widehat{R}_{\max}([b_1,b_2,\dots,b_{n-1}])\\
r_{2} &= \nu^2 + \begin{cases}
     0 & \text{if $\nu=n$}\\
    \widehat{R}_{\max}([b_{\nu},b_{\nu+1},\dots,b_{n-1}]) & \text{otherwise}
    \end{cases}
\end{aligned}
$$
We also have that $\widehat{R}([b_0])=0$ and we use the convention that $\widehat{R}([\;]) = -\infty$; we implicitly assume that the list $[b_{\nu},b_{\nu+1},\dots,b_{n-1}]$ is empty if $\nu\ge n$. The algorithm will terminate since the length of the list passed in the recursive call is reduced by at least one in each step.

This leads to the following recursive method for computing $\widehat{R}_{\max}(b)$:

#### Brute force Algorithm (non-circular version)
**input** $b=[b_0,b_1,\dots,b_{n-1}]$, a list of $n$ integer numbers, with $2\le b_j\le m$ for all $j=0,1,\dots,n-1$

**output** maximum reward $\widehat{R}_{\max}(b)$
1. **if** $b$ is empty
2. $~~~~$ **return** $-\infty$
3. **else if** $b$ contains exactly one element
4. $~~~~$ **return** $0$
5. **else**
6. $~~~~$ Set $r_1 = \widehat{R}_{\max}([b_1,b_2,\dots,b_{n-1}])$
7. $~~~~$ Set $r_2=\nu^2$ with $\nu:=b_0$
8. $~~~~$ **if** $\nu\ne n$
9. $~~~~~~~~$ Update $r_2 \mapsto r_2 + \widehat{R}_{\max}([b_{\nu},b_{\nu+1},\dots,b_{n-1}])$
10. $~~~~$ **end if**
11. $~~~~$ **return** $\max(r_1,r_2)$
12. **end if**

### Circular variant
The maximum award $R_{\max}(a)$ for the original game where the numbers $a_0,a_1,\dots,a_{n-1}$ are arranged in a circle can be obtained by considering all cyclic permutations $\mathring{a}^{(\omega)}$ of the list $a=[a_0,a_1,\dots,a_{n-1}]$ and computing the maximum reward $\widehat{R}(\mathring{a}^{(\omega)})$ in each case.

The cyclic permutation $\mathring{a}^{(\omega)}$ for $\omega\in\{0,1,2,\dots,n-1\}$ is obtained by shifting the numbers by $\omega$ to the left. For example if $a=[a_0,a_1,a_2,a_3,a_4,a_5]$ then

$$
\mathring{a}^{(1)} = [a_1,a_2,a_3,a_4,a_5,a_0]\;,\qquad\qquad
\mathring{a}^{(2)} = [a_2,a_3,a_4,a_5,a_0,a_1].
$$

### Divide and conquer algorithm
In the brute force implementation the number of recursive calls of $\widehat{R}_{\max}(b)$ will grow exponentially for large values of $n$. To overcome this issue, we can design a Divide-and-Conquer algorithm.

Again, we first consider the non-circular version of the game and define 
$$
n^* = \lfloor \frac{n}{2}\rfloor 
    = \begin{cases}
         \frac{n}{2} & \text{for $n$ even} \\[1ex]
         \frac{n-1}{2} & \text{for $n$ odd}.
      \end{cases}
$$


The crucial observation is that if $n>n^* + m - 1$, then the sequence of steps that leads to the maximum reward for the list $b=[b_0,b_1,b_2,\dots,b_{n-1}]$ will have to pass through one of the $m$ numbers $b_{n^*},b_{n^*+1},\dots,b_{n^*+m-1}$.

Once we have found a Divide-and-Conquer algorithm for the non-circular version of the game we can extend it to the original, circular version by considering cyclic permutations of the list of numbers.

## Assignment
In this coursework assignment you will write several Python functions for computing the maximum reward for the game described above.

Check that your code works by using the tests (see below). You will only receive full marks if your code passes those and similar tests. Comment your code. You will not receive full marks for code that is poorly commented or badly formatted.

Note that

* The assignment is split into five parts of increasing difficulty.
* **In your code you must use exactly the names of the functions and files given below. The same applies
to the ordering of parameters for these functions.**
* You can use any code you wrote in the Tickables, any code which has been made available as a model solution and all code which has been presented in the lectures.
* You must not copy code from any other sources, for example the internet. Doing so will be considered cheating.
* Further guidance on completing the coursework assignment can be found below

### Submission on Moodle
Please read the following instructions very carefully; it is your responsibility to submit your code correctly on
moodle.

All code must be submitted in a single notebook file called `Coursework_USERNAME.ipynb` where `USERNAME` is your username. For example, if your username is `gh3145`, then the name of the file should be `Coursework_gh3145.ipynb`. To create this file, please do the following:

1. Restart the kernel by clickiing on `Kernel` $\rightarrow$ `Restart & Clear Output` in the menu bar.
2. Execute the code in the following cell:

In [ ]:
!cp -r -n ${HOME}/ma12002.git/.templates/Coursework_template.ipynb ${HOME}/ma12002_workspace/

3. Rename the notebook `Coursework_template.ipynb` that has been created in your `ma12002_workspace` folder.

### Task 1a (implementation) $~~~~$ [7 points + 1 style point]
Implement the brute force algorithm described above by writing a **recursive** function `max_reward_noncircular(b)`, which computes the maximal reward $\widehat{R}_{\max}(b)$ for a given list $b=[b_0,b_1,\dots,b_{n-1}]$ in the **non-circular game**. Your function should return a single integer number and it should also return the correct result if the list $b$ is empty. Here is an example of how the function should be called:
```Python
b = [3,2,3,3,5,2,4]
Rhat_max = max_reward_noncircular(b)
```
After running this Python code, `Rhat_max` will have the value `18`.

*Hint: use `-numpy.infty` to represent $-\infty$* (minus infinity)

### Task 1b (numerical evaluation) $~~~~$ [1 point]
Use your code to compute the maximum possible reward for the following list:
```Python
[5, 6, 3, 4, 5, 2, 4, 3, 4, 5, 3, 7, 5, 7, 2, 2]
```

### Task 2 (cyclic permutations) $~~~~$ [4 points + 1 style point]
Write a Python function `cyclic_permutations(a)` which returns a list of all $n$ cyclic permutations of a given list $a=[a_0,a_1,\dots,a_{n-1}]$. For example, if $a=[1,4,2,7]$, then the function should be called as follows:
```Python
a = [1,4,2,7]
a_perm = cyclic_permutations(a)
```
After running this Python code, `a_perm` will be the list `[[1,4,2,7],[4,2,7,1],[2,7,1,4],[7,1,4,2]]`; note that each element of `a_perm`, is a list itself. The function `cyclic_permutations(a)` should return a list of lists even if $a$ contains only a single element; it should return an empty list if $a$ is empty. When implementing `cyclic_permutations(a)` you should not call any other functions.

### Task 3a (implementation) $~~~~$ [4 points + 1 style point]
Use the functions from Tasks 1 and 2 to write a function `max_reward(a)` which computes the maximum reward $R_{\max}(a)$ for a given list $a=[a_0,a_1,\dots,a_{n-1}]$ in the **original circular** game. Your code should also return the correct result if the list $a$ is empty. Here is an example of how the function should be called:
```Python
a = [2,2,4,3,2,2,2,3]
R_max = max_reward(a)
```
After running this Python code, `R_max` will have the value `25`.

### Task 3b (numerical evaluation)  $~~~~$ [1 point]
Use your code to compute the maximum possible reward for the following list (which is identical to the list in Task 1b):
```Python
[5, 6, 3, 4, 5, 2, 4, 3, 4, 5, 3, 7, 5, 7, 2, 2]
```

### Task 4a (DaQ implementation, non-circular game)  $~~~~$ [8 points + 1 style point]
Write a function `max_reward_daq_noncircular(b,m)` which implements a Divide-and-Conquer algorithm to compute $\widehat{R}_{\max}(b)$ for the **non-circular game.** The function should receive as inputs the list $b=[b_0,b_1,\dots,b_{n-1}]$ and the integer $m$ such that $2\le b_j\le m$. Make your function as efficient as possible. To avoid duplicating code, your function can call `max_reward_noncircular()` from Task 1, but you need to think carefully about where to do this without breaking the Divide-and-Conquer property of the algorithm. Make your function as efficient as possible, in the sense that the cost should grow at the smallest possible rate as the length $n$ of the list increases for $n\gg 1$ and a given value of $m$.

### Task 4b (DaQ implementation, circular game)  $~~~~$ [4 points + 1 style point]
Write a function `max_reward_daq(a)` which uses `max_reward_daq_noncircular(a,m)` to compute $R_{\max}(a)$ for the **original circular** game. The function `max_reward_daq(a)` should only receive the list $a=[a_0,a_1,\dots,a_{n-1}]$ as an input. Make your function as efficient as possible, in the sense that the cost should grow at the smallest possible rate as the length $n$ of the list increases for $n\gg 1$. Try to also minimise the growth in cost as $m$ increases.

*Hint: each sequence of steps will have to pass through the first $m$ numbers $a_0,a_1,\dots,a_{m-1}$*

### Task 5a (cost of recursive call) $~~~~$ [2 points]
Let $\widehat{E}(n,m)$ be the time it takes to execute `max_reward_daq_noncircular()` for a list of length $n$ with numbers in the range $2,3,\dots,m$. Assuming the optimal Divide-and-Conquer strategy has been used, which of the following statements about $\widehat{E}(n,m)$ is correct for $n\gg m>2$?

1. $\widehat{E}(n,m) \approx C_{1,1} + C_{1,2}m + 2 \widehat{E}(n/2,m)$
2. $\widehat{E}(n,m) \approx C_{2,1} + 2m \widehat{E}(n/2,m)$
3. $\widehat{E}(n,m) \approx C_{3,1} + C_{3,2}n + 2m \widehat{E}(n/2,m)$
4. $\widehat{E}(n,m) \approx C_{4,1} + m \widehat{E}(n/m,m)$
5. $\widehat{E}(n,m) \approx C_{5,1}n\cdot m + 2m \widehat{E}(n/2,m)$
6. $\widehat{E}(n,m) \approx C_{6,1} + 2\widehat{E}(n-m,m)$

(in these expressions all constants $C_{\alpha,\beta}>0$ are assumed to be positive, real numbers)

It is sufficient to write down the number of the correct statement. You do not need to justify your answer or provide derivations.

*Hint: $n^*\approx n^*+m-1 \approx n/2$ for $n\gg m>1$.*

### Task 5b (asymptotic complexity) $~~~~$ [4 points]
Let $E(n,m)$ be the time it takes to execute `max_reward_daq()` for an list of length $n$ with numbers in the range $2,3,\dots,m$. Assuming the optimal Divide-and-Conquer strategy has been used, and we are interested in the growth of the runtime as $n\gg 1$ for fixed $m>2$, which of the following statements are correct?

1. $E(n,m) = \Theta(n^{1/m})$
2. $E(n,m) = \Theta(n)$
3. $E(n,m) = \Theta(n\log_{2}(n))$
4. $E(n,m) = \Theta(n^2)$
5. $E(n,m) = \Theta(n^{\log_2(m)})$
6. $E(n,m) = \Theta(n\cdot n^{\log_2(m)})$
7. $E(n,m) = \Theta(n^m)$
8. $E(n,m) = \Theta(n\cdot 2^{nm})$

It is sufficient to write down the number of the correct statement(s). You do not need to justify your answer or provide derivations.

## Guidance
You might find the tests below useful, but you need to design additional tests yourself. A student who has actively engaged with the course and completed all tickables should be able to complete the assignment in 10-15
hours.

### Comments and style
Comment all your code in sufficient detail (see model solutions of tickables for examples), format it correctly
(indent code in if-statements and for-loops) and use sensible variable names. You will receive style points for well documented and formatted code. In particular, document all functions with docstrings. You need to make your own judgement as to how many comments to write. The key question you should ask yourself is: "can someone else understand the code based on the comments I wrote?". The [PEP 8 style guide](https://www.python.org/dev/peps/pep-0008/) might be helpful.

### Variable names
Avoid using `l` (lower case letter ''L'') as a variable name in your code since it looks very similar to `1` (the number 1). Spelling out the variable name as `ell` avoids any such confusion.

### Asking questions on the coursework
In the interest of fairness to all students I cannot reply to individual questions on this coursework, for example by email. Please ask any questions you might have in the weekly lecture on Thursdays or by posting on the [padlet discussion board](https://padlet.com/emueller16/ma12002-programming-xspfe6wlwphv97v4). If you have a question please also check this board since your question might already have been answered there.

There are certain questions that I will not answer, such as *"Can I do task X by doing Y?"* or *"Will I get more points if I do A in instead of B?"*. This is because answering such questions would give away the solution or risks misleading other students who read these questions.

You can work on the coursework during the computer lab session (but you should of course not work together).

### Avoiding computer problems
When working on this coursework, please take into account that computers don't always behave perfectly. This includes both your own computer, University PCs or the Notable Jupyter server. As a precaution, save your work at regular intervals and do not leave uploading to the last minute. You can also practice the upload procedure by working through Tickable 7, if you have not done so already. As an extra precaution you might want to download a copy of your work to your own computer at regular intervals.

### Notes on tutor guidance
The following describes what your lab tutors can, and cannot do, to help you with your coursework.
#### What does the coursework mean?
Tutors...
* ...can explain the mathematics to you
* ...cannot tell you whether your code is correct

#### Error messages
Tutors...
* can explain what a Python error message means;
* cannot tell you what to do to your program to make the message go away;
* may (depending on context) be able to ask you questions like *"how do you know..."* to help you work out what to do to your program.

#### Debugging
Tutors...
* can give you general guidance on strategies you might want to use to find errors in your code
* cannot fix your program for you
* can say "Why don’t you insert some print statement so you can see what the values actually are."
* may (depending on context) be able to ask you questions like *"is that variable's value what you expect"* to help you work out what’s going wrong.

#### How do I program this?
Tutors...
* cannot write your algorithms for you;
* may (depending on context) be able to say *"have you looked at the solution to tickable.../algorithm... from the lectures"*.

Tutors can...

* explain what a piece of Python (e.g. the `len()` function returns the length of a list) means in general (but not what it is doing in your code).
* say *"That looks like an extremely complicated way of doing XXX: why don’t you start again and look at the solution to tickable.../algorithm... from the lectures"*.

## Tests
You should carefully test your code and make sure that it works in all cases as discussed in the assignment. Copy the following tests to your notebook. The list is not exhaustive, so you might want to add your own tests to check your code. As always, you can run all tests with the `run_tests()` command.

In [ ]:
import numpy as np

####################################################
# TESTS
####################################################

#### Brute force, non-circular (Part 1a) ####

def test_max_reward_noncircular_three_elements_1():
    """Apply non-circular  brute force algorithm to list with three elements (case 1)"""
    assert max_reward_noncircular([3, 2, 4]) == 9

def test_max_reward_noncircular_three_elements_2():
    """Apply non-circular brute force algorithm to list with three elements (case 2)"""
    assert max_reward_noncircular([4, 2, 3]) == 4

def test_max_reward_noncircular_three_elements_3():
    """Apply non-circular brute force algorithm to list with three elements (case 3)"""
    assert max_reward_noncircular([4, 4, 4]) == 0
    
def test_max_reward_noncircular_ten_elements():
    """Check that non-circular brute force algorithm gives correct result for a list with ten elements"""
    assert max_reward_noncircular([3, 4, 6, 5, 3, 4, 5, 3, 6, 4]) == 36

#### Cyclic permutations (Part 2) ####

def test_cyclic_permutations_length_five():
    """Compute cyclic permutations of the list [0,1,2,3,4]"""
    assert cyclic_permutations([0, 1, 2, 3, 4]) == [
        [0, 1, 2, 3, 4],
        [1, 2, 3, 4, 0],
        [2, 3, 4, 0, 1],
        [3, 4, 0, 1, 2],
        [4, 0, 1, 2, 3],
    ]

#### Brute force (Part 3a) ####

def test_max_reward_three_elements_1():
    """Apply non-circular  brute force algorithm to list with three elements (case 1)"""
    assert max_reward([3, 2, 4]) == 9

def test_max_reward_ten_elements():
    """Brute force, non-circular, ten elements"""
    assert max_reward([3, 4, 6, 5, 3, 4, 5, 3, 6, 4]) == 45

#### Divide-and-Conquer, non-circular (Part 4a) ####

def test_max_reward_daq_noncircular_three_elements_1():
    """Apply non-circular Divide-and-Conquer algorithm to list with three elements (case 1)"""
    assert max_reward_daq_noncircular([3, 2, 4], 4) == 9

def test_max_reward_daq_noncircular_ten_elements():
    """Check that non-circular Divide-and-Conquer gives correct result on list with ten elements"""
    assert max_reward_daq_noncircular([3, 4, 6, 5, 3, 4, 5, 3, 6, 4], 6) == 36

def test_max_reward_daq_noncircular_large():
    """Check that the non-circular Divide-and-Conquer gives correct result on
    a very long list (n=60 elements,m=10)

    NB: if this times out, the Divide-and-Conquer implementation is probably wrong!
    """
    b = (
        [3, 2, 8, 10, 9, 10, 10, 4, 7, 5, 6, 4, 10, 9, 5, 7, 5, 10, 8, 2]
        + [10, 7, 9, 5, 7, 10, 9, 4, 9, 8, 7, 2, 10, 7, 2, 3, 6, 9, 5, 2]
        + [9, 4, 2, 5, 7, 4, 6, 6, 3, 3, 6, 5, 7, 6, 9, 2, 10, 8, 9, 6]
    )
    assert max_reward_daq_noncircular(b, 10) == 473

#### Divide-and-Conquer (Part 4b) ####

def test_max_reward_daq_three_elements_1():
    """Apply Divide-and-Conquer algorithm to list with three elements (case 1)"""
    assert max_reward_daq([3, 2, 4]) == 9

def test_max_reward_daq_ten_elements():
    """Check that Divide-and-Conquer gives correct results on list with ten elements"""
    assert max_reward_daq([3, 4, 6, 5, 3, 4, 5, 3, 6, 4]) == 45

def test_max_reward_daq_large():
    """Check that the Divide-and-Conquer algorithm gives correct results on
    a very long list (n=60 elements,m=10)

    NB: if this times out, the Divide-and-Conquer implementation is probably wrong!
    """
    b = (
        [3, 2, 8, 10, 9, 10, 10, 4, 7, 5, 6, 4, 10, 9, 5, 7, 5, 10, 8, 2]
        + [10, 7, 9, 5, 7, 10, 9, 4, 9, 8, 7, 2, 10, 7, 2, 3, 6, 9, 5, 2]
        + [9, 4, 2, 5, 7, 4, 6, 6, 3, 3, 6, 5, 7, 6, 9, 2, 10, 8, 9, 6]
    )
    assert max_reward_daq(b) == 498


run_tests()